In [9]:
# Note: if using a virtual environment, do not run this cell
#%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

'1.8.2'

### Semantic Kernel Test Notebook

This notebook is intended to ensure you have semantic kernel setup on your enviornment.
You should have a **.env** file that contains the following parameters that are used by semantic kernel

This was tested on version 1.8.2 of Semantic Kernel

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="XXX"
AZURE_OPENAI_ENDPOINT="YYYY"
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="gpt-4o-global"
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="gpt-4o-global"
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="text-embedding-ada-002"
AZURE_OPENAI_API_VERSION="2024-05-01-preview"
##Semantic Kernel Settings:
AZURE_AISEARCH_API_KEY=""
AZURE_AISEARCH_URL=""
```

In [10]:
#from services import Service
from service_settings import ServiceSettings
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import KernelArguments


from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_call_behavior import FunctionCallBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

from semantic_kernel.contents import ChatMessageContent, TextContent, ImageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

import os
from Plugin.EmailPlugin import EmailPlugin


### Setting up the Kernel
Set up the kernel and add a plug in from a class that is an "Email Plugin"

Its super basic.  The send_email function takes in a list of recieptns, subject and body.  
The kernel will be smart enough to use chat to get that information in order to send an email

```
from semantic_kernel.functions import kernel_function
from typing import List 

class EmailPlugin:
    @kernel_function(
        name="send_email",
        description="Sends an email to a recipient."
    )
    async def send_email(self, recipient_emails: str|List[str], subject: str, body: str):
        # Add logic to send an email using the recipient_emails, subject, and body
        # For now, we'll just print out a success message to the console
        print("Email sent!")
        print("recipient_emails:" + recipient_emails)
        print("subject:" + subject)
        print("body:" + body)
```

In [11]:

kernel = Kernel()
service_settings = ServiceSettings.create()
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
# selectedService = (
#     Service.AzureOpenAI
#     if service_settings.global_llm_service is None
#     else Service(service_settings.global_llm_service.lower())
# )
service_id = "default"
kernel.add_service(AzureChatCompletion(service_id=service_id,),)
kernel.add_plugin(EmailPlugin(),plugin_name="Email",)


KernelPlugin(name='Email', description=None, functions={'send_email': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='send_email', plugin_name='Email', description='Sends an email to a recipient.', parameters=[KernelParameterMetadata(name='recipient_emails', description=None, default_value=None, type_='str, list[str]', is_required=True, type_object=None, schema_data={'anyOf': [{'type': 'string'}, {'type': 'object'}]}, function_schema_include=True), KernelParameterMetadata(name='subject', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, function_schema_include=True), KernelParameterMetadata(name='body', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, function_schema_include=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_

Reference for Chat History:
https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/chat-history?pivots=programming-language-python

In [12]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior


chat_completion : AzureChatCompletion = kernel.get_service(type=ChatCompletionClientBase)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings(tool_choice="auto")
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()
# Create a history of the conversation
history = ChatHistory()
history.add_system_message("You are a helpful assistant.")
history.add_user_message("Can you help me write an email for my boss?")

result = (await chat_completion.get_chat_message_contents(
        chat_history=history,
        settings=execution_settings,
        kernel=kernel,
        arguments=KernelArguments(),
    ))[0]
print(result)


history.add_assistant_message(str(result))


Of course, I'd be happy to help you write an email to your boss. To get started, I'll need some information:

1. The subject of the email: What is the main topic or the purpose of the email?
2. The content: What specific details do you want to include in the body of the email?
3. Tone: Should the email be formal, casual, apologetic, urgent, etc.?
4. Any other preferences or special requests: Is there something specific that needs to be mentioned or a particular way you want to structure the email?

Please provide me with these details, and then I can help you craft the email.


In [16]:
for hist in history:
    print(hist.role)
    print(hist.content)

AuthorRole.SYSTEM
You are a helpful assistant.
AuthorRole.USER
Can you help me write an email for my boss?
AuthorRole.ASSISTANT
Of course, I'd be happy to help you write an email to your boss. To get started, I'll need some information:

1. The subject of the email: What is the main topic or the purpose of the email?
2. The content: What specific details do you want to include in the body of the email?
3. Tone: Should the email be formal, casual, apologetic, urgent, etc.?
4. Any other preferences or special requests: Is there something specific that needs to be mentioned or a particular way you want to structure the email?

Please provide me with these details, and then I can help you craft the email.


In [5]:


history.add_user_message("subject: Azure is Super Cool, main points: azure ai is very powerful.  My boss doesn't need to do anything, I've got this.  Use a semi-formal tone.")

result = (await chat_completion.get_chat_message_contents(
        chat_history=history,
      
        settings=execution_settings,
        kernel=kernel,
        arguments=KernelArguments(),
    ))[0]
print(result)


history.add_assistant_message(str(result))

Certainly! Here's a draft for your email with the information you provided:

Subject: Azure is Super Cool

Dear [Boss's Name],

I hope this message finds you well. I wanted to take a moment to share my excitement about the capabilities we have at our fingertips with Azure AI.

As I've delved into our latest project, it's become increasingly clear how powerful Azure's AI services are. The suite of tools available, ranging from machine learning models to cognitive services, has not only streamlined our development process but also opened up innovative avenues for us to explore.

I am confident that Azure AI will not only meet but exceed our project objectives with its robust features and seamless integration potential. It is impressive how it simplifies complex tasks, facilitates predictive analytics, and enables us to harness the power of big data with ease.

I want to assure you that everything is progressing smoothly. You can rest easy knowing that I have taken the reins of our Azure 

In [6]:
history.add_user_message("ok, boss's email: boss@gmail.com, please send it to her.")

result = (await chat_completion.get_chat_message_contents(
        chat_history=history,
      
        settings=execution_settings,
        kernel=kernel,
        arguments=KernelArguments(),
    ))[0]
print(result)

history.add_assistant_message(str(result))

Email sent!
recipient_emails:boss@gmail.com
subject:Azure is Super Cool
body:Dear [Boss's Name],

I hope this message finds you well. I wanted to take a moment to share my excitement about the capabilities we have at our fingertips with Azure AI.

As I've delved into our latest project, it's become increasingly clear how powerful Azure's AI services are. The suite of tools available, ranging from machine learning models to cognitive services, has not only streamlined our development process but also opened up innovative avenues for us to explore.

I am confident that Azure AI will not only meet but exceed our project objectives with its robust features and seamless integration potential. It is impressive how it simplifies complex tasks, facilitates predictive analytics, and enables us to harness the power of big data with ease.

I want to assure you that everything is progressing smoothly. You can rest easy knowing that I have taken the reins of our Azure AI integration, and I am monit

### Moving on...
We can ask for a joke, but this is not using the joke template taht we did in 001_tester.ipynb

In [7]:
history.add_user_message("tell me a joke")

result = (await chat_completion.get_chat_message_contents(
        chat_history=history,
      
        settings=execution_settings,
        kernel=kernel,
        arguments=KernelArguments(),
    ))[0]
print(result)
history.add_assistant_message(str(result))

Sure, here's a light-hearted tech joke for you:

Why don't programmers like nature?

It has too many bugs!


In [8]:
for msg in history.messages:
    print(msg.role + ":" + msg.content)

system:You are a helpful assistant.
user:Can you help me write an email for my boss?
assistant:Of course! I'd be happy to help you draft an email for your boss. However, before we begin, could you provide me with the following details?

1. The purpose of the email: Is it to report progress, ask for feedback, request time off, etc.?
2. Key points you want to include: Any specific information or bullet points that should be mentioned in the email.
3. The tone of the email: Formal, informal, persuasive, apologetic, etc.
4. Any specific phrases or sentences you want to be included in the email.
5. Your boss's name and email address (if we are also sending the email).

Once you provide this information, I can help you compose a suitable email. If you prefer not to disclose your boss's email address, I can still draft the email, and then you can send it yourself. Let me know how you'd like to proceed!
user:subject: Azure is Super Cool, main points: azure ai is very powerful.  My boss doesn't